# First Lookup on DemoParse Library


In [1]:
from demoparser2 import DemoParser
import pandas as pd
from pprint import pprint
from typing import Dict, Sequence, Optional, List, Tuple
import numpy as np

In [2]:
base_path = "../../demos"
parser = DemoParser(f"{base_path}/faceit/anubisFaceit.dem")

event_names = parser.list_game_events()

# Currently the event "all" gives you all events. Cursed solution for now
event_names

['inferno_expire',
 'round_announce_match_start',
 'item_pickup',
 'round_announce_warmup',
 'player_blind',
 'round_officially_ended',
 'hegrenade_detonate',
 'bomb_begindefuse',
 'item_equip',
 'weapon_fire',
 'server_message',
 'decoy_started',
 'player_hurt',
 'decoy_detonate',
 'weapon_zoom',
 'chat_message',
 'player_spawn',
 'player_team',
 'player_connect',
 'announce_phase_end',
 'cs_round_final_beep',
 'bomb_exploded',
 'cs_win_panel_match',
 'show_survival_respawn_status',
 'player_footstep',
 'smokegrenade_detonate',
 'bomb_beginplant',
 'round_announce_last_round_half',
 'hltv_chase',
 'round_announce_final',
 'smokegrenade_expired',
 'cs_round_start_beep',
 'round_time_warning',
 'round_prestart',
 'round_freeze_end',
 'hltv_fixed',
 'bomb_planted',
 'player_connect_full',
 'server_cvar',
 'begin_new_match',
 'player_jump',
 'bomb_dropped',
 'cs_pre_restart',
 'weapon_reload',
 'round_poststart',
 'buytime_ended',
 'player_death',
 'inferno_startburn',
 'bomb_defused',
 '

# Game Score

Halftime or final score


In [3]:
def get_player_average_damage_per_round():
    """
    Function to calculate the Average Damage per Round (ADR) for all players.

    Args:
        parser (DemoParser): An instance of DemoParser initialized with a demo file path.

    Returns:
        pd.DataFrame: A dataframe containing player Steam IDs and their ADR.
    """
    # Define the fields to extract
    wanted_fields = ["damage_total", "kills_total", "deaths_total"]

    # Parse the number of rounds
    round_end_df = parser.parse_event("round_end")
    return len(round_end_df) - 1  # Count the total number of rounds

    # # Parse the maximum tick to analyze final stats
    # max_tick = round_end_df["tick"].max()

    # # Parse the wanted fields for all players at the last tick
    # stats_df = parser.parse_ticks(wanted_fields, ticks=[max_tick])

    # # Ensure no NaN values in the stats
    # stats_df.fillna(0, inplace=True)

In [4]:
def get_final_score(
    players: Sequence[str] = None,
    round_info: str | int = "final",
) -> Tuple[pd.DataFrame, pd.DataFrame] | pd.DataFrame:
    """
    Retrieve the final score details for specified players or all players at the end of a round.

    Parameters:
    -----------
    players : Sequence[str], optional
        A list of player names to filter the DataFrame. If provided, the returned DataFrame
        will only include data for the specified players. Defaults to None.

    round_info : str | int, optional
        Specifies the round for which to retrieve score data. Can take the following values:
        - "final" (default): Retrieves the final round score.
        - "half_time": Retrieves the score at halftime (end of round 12).
        - An integer: Retrieves the score for the specified round (e.g., 1 for the first round).
        If the integer is greater than the maximum round available, a ValueError is raised.

    Returns:
    --------
    Union[Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame], pd.DataFrame]
        - If `players` is provided: A single filtered DataFrame containing data for the specified players.
        - Otherwise: A tuple containing:
            1. DataFrame for Team 1
            2. DataFrame for Team 2
            3. DataFrame containing data for all players, sorted by team and KD ratio.

    Raises:
    -------
    ValueError
        - If `round_info` is not "final", "half_time", or a valid integer.
        - If `round_info` is an integer greater than the maximum round available.
    """

    # Determine the tick based on round_info
    last_tick = parser.parse_event("round_end")['tick'].max()

    events = pd.concat([
        parser.parse_event("round_officially_ended")["tick"].drop_duplicates(),
        pd.Series([last_tick])
    ], ignore_index=True)

    events.index = range(1, len(events) + 1)

    max_round = len(events)

    # Validate and process `round_info`
    special_rounds = {
    "half_time": 12,
    "final": max_round
    }

    # Validate and process `round_info`
    if isinstance(round_info, int):
        if not 1 <= round_info <= max_round:
            raise ValueError(f"Invalid `round_info`: {round_info}. Maximum round is {max_round}.")
    elif isinstance(round_info, str) and round_info in special_rounds:
        round_info = special_rounds[round_info]
    else:
        raise ValueError("Invalid `round_info`. Must be 'final', 'half_time', or an integer.")


    # Get the tick corresponding to `round_info`
    tick = events.loc[round_info]

    wanted_fields = [
        "kills_total",
        "deaths_total",
        "mvps",
        "headshot_kills_total",
        "ace_rounds_total",
        "4k_rounds_total",
        "3k_rounds_total",
        "team_num",
        "damage_total",
        "assists_total",
        "team_score_first_half",
        "team_score_second_half",
    ]

    # Parse the ticks at the max_tick
    df = parser.parse_ticks(wanted_fields, ticks=[tick])
    df["deaths_total"] = df["deaths_total"].fillna(0)

    # Calculate KD
    df["kd"] = np.where(
        df["deaths_total"] != 0,
        round(df["kills_total"] / df["deaths_total"], 2),
        round((df["kills_total"] / 1), 2),
    )

    # Calculate HS %
    df["headshot_percentage"] = np.where(
        df["kills_total"] != 0,
        round(df["headshot_kills_total"] / df["kills_total"] * 100),
        round((df["headshot_kills_total"] / 1) * 100),
    ).astype(int)

    # Calculate ADR
    print(df["damage_total"])
    df["adr"] = round(df["damage_total"] / round_info, 2)
    df["kpr"] = round(df["kills_total"] / round_info, 2)
    df["dpr"] = round(df["deaths_total"] / round_info, 2)
    
    df["round"] = round_info

    df["diff"] = df["kills_total"] - df["deaths_total"]

    df.sort_values("adr", inplace=True, ascending=False)

    # If specific players are provided, filter the DataFrame and return
    if players:
        df = df[df["name"].isin(players)]
        return df

    # Get unique team numbers dynamically
    unique_teams = df["team_num"].unique()

    # Assign the team numbers dynamically
    team_num_1, team_num_2 = unique_teams[:2]

    # Create separate DataFrames for each team
    df_team_1: pd.DataFrame = df[df["team_num"] == team_num_1].copy()
    df_team_2: pd.DataFrame = df[df["team_num"] == team_num_2].copy()

    return df_team_1, df_team_2, df


df = get_final_score(round_info=23)
df[2]

0    1789
1    2102
2    1457
3    1572
4    1848
5    1826
6    2517
7    1665
8    1978
9    1146
Name: damage_total, dtype: int32


,kills_total,deaths_total,assists_total,headshot_kills_total,damage_total,ace_rounds_total,4k_rounds_total,3k_rounds_total,mvps,team_num,...,tick,steamid,name,kd,headshot_percentage,adr,kpr,dpr,round,diff
6,24,17,5,15,2517,0,0,2,5,3,...,166261,76561199075107764,PuliNFPS,1.41,62,109.43,1.04,0.74,23,7
1,21,14,4,11,2102,0,0,1,2,2,...,166261,76561198840306307,ton001,1.50,52,91.39,0.91,0.61,23,7
8,15,19,4,10,1978,0,0,1,2,2,...,166261,76561198057428801,snowwJL,0.79,67,86.00,0.65,0.83,23,-4
4,18,14,5,5,1848,0,0,1,3,3,...,166261,76561198317791204,MIRAcls_,1.29,28,80.35,0.78,0.61,23,4
5,18,18,2,12,1826,0,0,1,3,2,...,166261,76561197987161730,sr_jarbolas,1.00,67,79.39,0.78,0.78,23,0
0,18,13,3,9,1789,0,0,3,1,3,...,166261,76561198803359038,EvUsFDL,1.38,50,77.78,0.78,0.57,23,5
7,12,17,13,4,1665,0,0,0,1,3,...,166261,76561198437803219,creepyfps,0.71,33,72.39,0.52,0.74,23,-5
3,16,13,5,8,1572,0,0,2,2,3,...,166261,76561198077572982,cebola1_,1.23,50,68.35,0.70,0.57,23,3
2,10,20,6,5,1457,0,0,0,2,2,...,166261,76561197999127404,vap999,0.50,50,63.35,0.43,0.87,23,-10
9,10,18,4,4,1146,0,0,1,2,2,...,166261,76561198129776170,brbzinN,0.56,40,49.83,0.43,0.78,23,-8


In [8]:
wanted_fields = [
        "kills_total",
        "deaths_total",
        "assists_total",
        "damage_total",
        "team_score_first_half",
        "team_score_second_half",
    ]

    # Parse the ticks at the max_tick
ticks = parser.parse_event("round_end")["tick"]
df = parser.parse_ticks(wanted_fields, ticks=[165813])
df
# for tick in ticks:
    
#     df['tick'] = tick
#     df = df[df['steamid'] == 76561199075107764]
#     print(df)


,kills_total,deaths_total,assists_total,damage_total,team_score_first_half,team_score_second_half,tick,steamid,name
0,18,13,3,1789,8,4,165813,76561198803359038,EvUsFDL
1,21,14,4,2073,4,7,165813,76561198840306307,ton001
2,10,20,6,1397,4,7,165813,76561197999127404,vap999
3,16,13,5,1472,8,4,165813,76561198077572982,cebola1_
4,18,14,5,1707,8,4,165813,76561198317791204,MIRAcls_
5,18,18,2,1815,4,7,165813,76561197987161730,sr_jarbolas
6,24,17,5,2313,8,4,165813,76561199075107764,PuliNFPS
7,12,17,13,1610,8,4,165813,76561198437803219,creepyfps
8,15,19,4,1978,4,7,165813,76561198057428801,snowwJL
9,10,18,4,1132,4,7,165813,76561198129776170,brbzinN


In [6]:
def get_final_score(
    players: Sequence[str] = None,
    round_info: str | int = "final",
) -> Tuple[pd.DataFrame, pd.DataFrame] | pd.DataFrame:
    """
    Retrieve the final score details for specified players or all players at the end of a round.

    Parameters:
    -----------
    players : Sequence[str], optional
        A list of player names to filter the DataFrame. If provided, the returned DataFrame
        will only include data for the specified players. Defaults to None.

    round_info : str | int, optional
        Specifies the round for which to retrieve score data. Can take the following values:
        - "final" (default): Retrieves the final round score.
        - "half_time": Retrieves the score at halftime (end of round 12).
        - An integer: Retrieves the score for the specified round (e.g., 1 for the first round).
        If the integer is greater than the maximum round available, a ValueError is raised.

    Returns:
    --------
    Union[Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame], pd.DataFrame]
        - If `players` is provided: A single filtered DataFrame containing data for the specified players.
        - Otherwise: A tuple containing:
            1. DataFrame for Team 1
            2. DataFrame for Team 2
            3. DataFrame containing data for all players, sorted by team and KD ratio.

    Raises:
    -------
    ValueError
        - If `round_info` is not "final", "half_time", or a valid integer.
        - If `round_info` is an integer greater than the maximum round available.
    """

    # Determine the tick based on round_info
    last_tick = parser.parse_event("round_end")
    events = parser.parse_event("cs_round_final_beep")['tick']
    
    max_round = len(events) - 1
    
    if isinstance(round_info, int):
        if round_info > max_round:
            raise ValueError(
                f"Invalid `round_info`: {round_info}. Maximum round is {max_round}."
            )
        tick = events.iloc[round_info]["tick"]
    elif round_info == "half_time":
        round_info = 12
        tick = events.iloc[round_info]["tick"]["tick"].max()
    elif round_info == "final":
        round_info = max_round
        tick = events["tick"].max()
    else:
        raise ValueError(
            "Invalid `round_info` value. Must be 'final', 'half_time', or an integer."
        )

    wanted_fields = [
        "kills_total",
        "deaths_total",
        "mvps",
        "headshot_kills_total",
        "ace_rounds_total",
        "4k_rounds_total",
        "3k_rounds_total",
        "team_num",
        "damage_total",
        "assists_total",
        "team_score_first_half",
        "team_score_second_half",
    ]

    # Parse the ticks at the max_tick
    df = parser.parse_ticks(wanted_fields, ticks=[tick])
    df["deaths_total"] = df["deaths_total"].fillna(0)

    # Calculate KD
    df["kd"] = np.where(
        df["deaths_total"] != 0,
        round(df["kills_total"] / df["deaths_total"], 2),
        round((df["kills_total"] / 1), 2),
    )

    # Calculate HS %
    df["headshot_percentage"] = np.where(
        df["kills_total"] != 0,
        round(df["headshot_kills_total"] / df["kills_total"] * 100),
        round((df["headshot_kills_total"] / 1) * 100),
    ).astype(int)

    # Calculate ADR
    print(df["damage_total"])
    df["adr"] = round(df["damage_total"] / round_info, 2)
    df["kpr"] = round(df["kills_total"] / round_info, 2)
    df["dpr"] = round(df["deaths_total"] / round_info, 2)
    
    df["round"] = round_info

    df["diff"] = df["kills_total"] - df["deaths_total"]
    
    # df.drop(columns=["tick"], inplace=True)

    df.sort_values("adr", inplace=True, ascending=False)

    # If specific players are provided, filter the DataFrame and return
    if players:
        df = df[df["name"].isin(players)]
        return df

    # Get unique team numbers dynamically
    unique_teams = df["team_num"].unique()

    # Assign the team numbers dynamically
    team_num_1, team_num_2 = unique_teams[:2]

    # Create separate DataFrames for each team
    df_team_1: pd.DataFrame = df[df["team_num"] == team_num_1].copy()
    df_team_2: pd.DataFrame = df[df["team_num"] == team_num_2].copy()

    return df_team_1, df_team_2, df


df = get_final_score(round_info=1)
df[2]

IndexError: invalid index to scalar variable.

In [141]:
last_tick = parser.parse_event("round_end")['tick'].max()
events = parser.parse_event("round_officially_ended")["tick"].drop_duplicates().reset_index(drop=True)
events = pd.concat([events, pd.Series([last_tick])], ignore_index=True)
events.index = range(1, len(events) + 1)

print(events)

1       8691
2      13749
3      20996
4      28586
5      36078
6      44175
7      49495
8      57404
9      65264
10     70633
11     83127
12     87817
13     93618
14    101144
15    109754
16    117025
17    126970
18    134956
19    141341
20    150126
21    156607
22    162096
23    166261
24    172193
dtype: int32
